# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classdb', 'config', 'epa', 'fruits_DB', 'gardendb', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
print(db.establishments.find_one())

{'_id': ObjectId('6473f8dbe5eb945d8071a853'), 'FHRSID': 647177, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000041489', 'BusinessName': 'Wear Bay Bowls Club', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'Wear Bay Road', 'AddressLine2': 'Folkestone', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT19 6PY', 'Phone': '', 'RatingValue': 4, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2014-03-31T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'}, 'RightToReply': '', 'Distance': 4591.821311183521, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': Non

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [8]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Find the establishments with hygiene score 20 and display the first document
result = establishments.find(query)

# Use count_documents to display the number of documents in the result
hygiene_score = establishments.count_documents(query)

# Use pprint to display the value
pprint(db.establishments.find_one(query))

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6473f8dbe5eb945d8071c388'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'longitude': None,
 'meta': 

In [14]:
import pandas as pd

# Convert mongo result to Pandas DataFrame
result_df = pd.DataFrame(list(establishments.find(query)))

row_count = result_df._id.count()
#result_df.head(10)
row_count

164

In [ ]:
print(result_df.columns)

Index(['_id', 'FHRSID', 'ChangesByServerID', 'LocalAuthorityBusinessID',
       'BusinessName', 'BusinessType', 'BusinessTypeID', 'AddressLine1',
       'AddressLine2', 'AddressLine3', 'AddressLine4', 'PostCode', 'Phone',
       'RatingValue', 'RatingKey', 'RatingDate', 'LocalAuthorityCode',
       'LocalAuthorityName', 'LocalAuthorityWebSite',
       'LocalAuthorityEmailAddress', 'scores', 'SchemeType', 'geocode',
       'RightToReply', 'Distance', 'NewRatingPending', 'meta', 'links',
       'latitude', 'longitude'],
      dtype='object')


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [41]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4
query1 = {'$match': {'HygieneScore': 20}}

query2 = {
    '$match': {
        'LocalAuthorityName': {'$regex': 'London', '$options': 'i'},
        'RatingValue': {'$gte': 4}
    }
}

sort_values = {'$sort': {'count': -1, '_id': 1}}

pipeline = [query1, query2, sort_values]

results = list(establishments.aggregate(pipeline))

print("Number of establishments in result:", len(results))
pprint(results[0:10])


# Find the establishments matching the query
result1 = establishments.find(query1)

# Use pprint to display the values
#pprint(first_document1['RatingValue'])


# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of establishments with London as Local Authority and RatingValue >= 4:", count)


# Display the first document in the results using pprint
first_document1 = next(result1)
pprint.pprint(first_document1)

Number of establishments in result: 0
[]
Number of establishments with London as Local Authority and RatingValue >= 4: 164


OperationFailure: unknown top level operator: $match. If you have a field name that starts with a '$' symbol, consider using $getField or $setField., full error: {'ok': 0.0, 'errmsg': "unknown top level operator: $match. If you have a field name that starts with a '$' symbol, consider using $getField or $setField.", 'code': 2, 'codeName': 'BadValue'}

In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [13]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 
longitude = 

query = 
sort =  

# Print the results


SyntaxError: invalid syntax (848538274.py, line 6)

In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
